# IHLT Lab 5: Lexical Semantics

**Authors:** *Zachary Parent ([zachary.parent](mailto:zachary.parent@estudiantat.upc.edu)), Carlos Jiménez ([carlos.humberto.jimenez](mailto:carlos.humberto.jimenez@estudiantat.upc.edu))*

### 2024-10-17

**Instructions:**

- Given the following (lemma, category) pairs:

`(’the’,’DT’), (’man’,’NN’), (’swim’,’VB’), (’with’, ’PR’), (’a’, ’DT’),
(’girl’,’NN’), (’and’, ’CC’), (’a’, ’DT’), (’boy’, ’NN’), (’whilst’, ’PR’),
(’the’, ’DT’), (’woman’, ’NN’), (’walk’, ’VB’)`

- For each pair, when possible, print their most frequent WordNet synset

- For each pair of words, when possible, print their corresponding least common subsumer (LCS) and their similarity value, using the following functions:

    - Path Similarity

    - Peacock-Chodorow Similarity

    - Wu-Palmer Similarity

    - Lin Similarity

Normalize similarity values when necessary. What similarity seems better?


## Setup

In [6]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.corpus import sentiwordnet as swn

In [7]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('sentiwordnet')
nltk.download('wordnet_ic')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/carlos.jimenez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/carlos.jimenez/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/carlos.jimenez/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /home/carlos.jimenez/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


True

In [8]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')

In [11]:
def getRelations(ss):
    lexRels = ['hypernyms', 'instance_hypernyms', 'hyponyms', \
       'instance_hyponyms', 'member_holonyms', 'substance_holonyms', \
       'part_holonyms', 'member_meronyms', 'substance_meronyms', \
       'part_meronyms', 'attributes', 'entailments', 'causes', 'also_sees', \
       'verb_groups', 'similar_tos']
    def getRelValue(ss, name):
        method = getattr(ss, rel)
        return method()
    
    results = {}
    for rel in lexRels:
        val = getRelValue(ss, rel)
        if val != []:
            results[rel] = val
    return results

## WordNet Similarities

In [17]:
dog.lowest_common_hypernyms(cat) 

[Synset('carnivore.n.01')]

In [18]:
dog.path_similarity(cat)

0.2

In [19]:
dog.lch_similarity(cat) 


2.0281482472922856

In [20]:
dog.wup_similarity(cat) 

0.8571428571428571

In [21]:
brown_ic = wordnet_ic.ic('ic-brown.dat')
dog.lin_similarity(cat,brown_ic)

0.8768009843733973

## SentiWordnet 

In [22]:
# getting the wordnet synset
synset = wn.synset('good.a.1')
# getting the sentiwordnet synset
sentiSynset = swn.senti_synset(synset.name())

sentiSynset.pos_score(), sentiSynset.neg_score(), sentiSynset.obj_score()

(0.75, 0.0, 0.25)

In [ ]:
words = [('the','DT'), ('man','NN'), ('swim','VB'), ('with', 'PR'), ('a', 'DT'),
('girl','NN'), ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), ('whilst', 'PR'),
('the', 'DT'), ('woman', 'NN'), ('walk', 'VB')]